In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import numpy as np
import scipy as sp
import os
import matplotlib.pyplot as plt

module_path = "../../ad_afqmc"
module_path = "../../projected_mf"

if module_path not in sys.path:
    sys.path.append(module_path)

from ad_afqmc import (
    driver,
    pyscf_interface,
    mpi_jax,
    linalg_utils,
    spin_utils,
    lattices,
    propagation,
    wavefunctions,
    hamiltonian,
    hf_guess
)

from projected_ghf import (
    get_wigner_d, 
    apply_Sz_projector,
    apply_Sz_projector_jax,
    get_real_wavefunction,
    get_energy,
    get_energy_jax,
    get_projected_energy,
    get_projected_energy_jax,
    gradient_descent,
    optimize
)

from pyscf import fci, gto, scf, mp, ao2mo
import jax.numpy as jnp

np.set_printoptions(precision=5, suppress=True)

# Hostname: g234
# System Type: Linux
# Machine Type: x86_64
# Processor: x86_64


# Test `get_wigner_d`

In [3]:
def test_get_wigner_d():
    beta = np.random.random() * np.pi
    
    s, m, k = 0, 0, 0
    ref = 1.
    np.testing.assert_allclose(ref, get_wigner_d(s, m, k, beta))
    
    s, m, k = 0.5, 0.5, 0.5
    ref = np.cos(beta/2.)
    np.testing.assert_allclose(ref, get_wigner_d(s, m, k, beta))
    
    s, m, k = 0.5, 0.5, -0.5
    ref = -np.sin(beta/2.)
    np.testing.assert_allclose(ref, get_wigner_d(s, m, k, beta))
    
    s, m, k = 1, 1, 1
    ref = (1. + np.cos(beta)) / 2.
    np.testing.assert_allclose(ref, get_wigner_d(s, m, k, beta))
    
    s, m, k = 1, 1, 0
    ref = -np.sin(beta) / np.sqrt(2.)
    np.testing.assert_allclose(ref, get_wigner_d(s, m, k, beta))
    
    s, m, k = 1, 1, -1
    ref = (1. - np.cos(beta)) / 2.
    np.testing.assert_allclose(ref, get_wigner_d(s, m, k, beta))
    
    s, m, k = 1.5, 1.5, 1.5
    ref = (1. + np.cos(beta)) * np.cos(beta/2.) / 2.
    np.testing.assert_allclose(ref, get_wigner_d(s, m, k, beta))
    
    s, m, k = 2, 1.5, 1.5
    ref = (1. + np.cos(beta))**2 / 4.
    np.testing.assert_allclose(ref, get_wigner_d(s, m, k, beta))

In [4]:
test_get_wigner_d()

# Test `apply_Sz_projector`

# Test with $|+\rangle = \frac{1}{\sqrt{2}}(|\uparrow\rangle + |\downarrow\rangle)$

In [3]:
def get_greens(bra, ket, verbose=False):
    norb = bra.shape[0] // 2
    omat = bra.T.conj() @ ket
    ovlp = np.linalg.det(omat)
    greens = np.zeros((2*norb, 2*norb))
    
    if np.absolute(ovlp) < 1e-15:
        if verbose: print(f'ovlp = {ovlp}')
        return greens, ovlp
    
    greens = ket @ np.linalg.inv(omat) @ bra.T.conj()
    return greens.T, ovlp

In [4]:
norb = 3
nocc = 1
psi = np.array([[1., 0., 0.], 
                [0., 1., 0.], 
                [0., 0., 1.], 
                [1., 0., 0.], 
                [0., 1., 0.], 
                [0., 0., 1.]]) / np.sqrt(2.)

In [5]:
# Initial spin average and overlap.
G, ovlp = get_greens(psi[:, :nocc], psi[:, :nocc])
savg = spin_utils.get_spin_average_from_greens_ghf(G) * ovlp

print(f'<S> = {savg}')
print(f'ovlp = {ovlp}')

<S> = [0.5 0.  0. ]
ovlp = 0.9999999999999998


I0000 00:00:1736537621.637797  405499 tfrt_cpu_pjrt_client.cc:349] TfrtCpuClient created.


## Project to $s_z = 1/2$

In [24]:
kets, coeffs = apply_Sz_projector_jax(psi[:, :nocc], 0.5, 500) # Should just give |u>.
kets = np.array(kets)
coeffs = np.array(coeffs)

In [25]:
# Projected spin average and overlap.
savg_tot = np.zeros(3, dtype=np.complex128)
ovlp_tot = 0.

for ibra, bra in enumerate(kets):
    for iket, ket in enumerate(kets):
        G, ovlp = get_greens(bra, ket)
        savg = spin_utils.get_spin_average_from_greens_ghf(G) * ovlp
        savg_tot += savg * coeffs[ibra].conj() * coeffs[iket]
        ovlp_tot += ovlp * coeffs[ibra].conj() * coeffs[iket]

In [26]:
print(f'<S> * ovlp = {savg_tot}')
print(f'ovlp = {ovlp_tot}')
print(f'\n<S> = {savg_tot / ovlp_tot}')

<S> * ovlp = [-0.   -0.j -0.   +0.j  0.249-0.j]
ovlp = (0.5000000000000908+1.5119935155183287e-18j)

<S> = [-0.   -0.j -0.   +0.j  0.498-0.j]


## Project to $s_z = -1/2$

In [27]:
kets, coeffs = apply_Sz_projector_jax(psi[:, :nocc], -0.5, 500) # Should just give |d>.
kets = np.array(kets)
coeffs = np.array(coeffs)

In [28]:
# Projected spin average and overlap.
savg_tot = np.zeros(3, dtype=np.complex128)
ovlp_tot = 0.

for ibra, bra in enumerate(kets):
    for iket, ket in enumerate(kets):
        G, ovlp = get_greens(bra, ket)
        savg = spin_utils.get_spin_average_from_greens_ghf(G) * ovlp
        savg_tot += savg * coeffs[ibra].conj() * coeffs[iket]
        ovlp_tot += ovlp * coeffs[ibra].conj() * coeffs[iket]

In [29]:
print(f'<S> * ovlp = {savg_tot}')
print(f'ovlp = {ovlp_tot}')
print(f'\n<S> = {savg_tot / ovlp_tot}')

<S> * ovlp = [-0.   +0.j -0.   -0.j -0.249-0.j]
ovlp = (0.5000000000000908-1.5119935155183287e-18j)

<S> = [-0.   +0.j -0.   -0.j -0.498-0.j]


## Project to $s_z = 0$

We apply the projector 
\begin{align}
    \hat{P}_0 |+\rangle &= \frac{1}{\sqrt{2}} \frac{1}{2\pi} \int_0^{2\pi} e^{i\phi\hat{S}_z} d\phi \left( |\uparrow\rangle + |\downarrow\rangle \right) \\
    &= \frac{1}{\sqrt{2}} \cdot \frac{1}{2\pi} \left\{ \int_0^{2\pi} e^{i\phi\hat{S}_z} |\uparrow\rangle d\phi + \int_0^{2\pi} e^{i\phi\hat{S}_z} d\phi |\downarrow\rangle d\phi \right\} \\
    &= \frac{1}{\sqrt{2}} \cdot \frac{1}{2\pi} \left\{ \int_0^{2\pi} e^{i\phi/2} |\uparrow\rangle d\phi + \int_0^{2\pi} e^{-i\phi/2} d\phi |\downarrow\rangle d\phi \right\} \\
    &= \frac{1}{\sqrt{2}} \cdot \frac{1}{2\pi} \left\{ \left[\frac{2}{i} e^{i\phi/2}\right]_0^{2\pi} |\uparrow\rangle + \left[-\frac{2}{i} e^{-i\phi/2}\right]_0^{2\pi} |\downarrow\rangle  \right\} \\
    &= \frac{1}{\sqrt{2}} \cdot \frac{1}{2\pi} \left\{ \frac{2}{i} \left(e^{i\phi} - 1\right) |\uparrow\rangle + -\frac{2}{i} \left(e^{-i\phi} - 1\right) |\downarrow\rangle  \right\} \\
    &= \frac{1}{\sqrt{2}} \cdot \frac{1}{2\pi} \left\{ -\frac{4}{i} |\uparrow\rangle + \frac{4}{i} |\downarrow\rangle  \right\} \\
    &= \frac{1}{\sqrt{2}} \cdot \frac{2i}{\pi} \left\{ |\uparrow\rangle - |\downarrow\rangle  \right\} \\
    &= \frac{2i}{\pi} |-\rangle,
\end{align}

which gives
\begin{align}
    \langle \hat{S}_x \rangle &= \left(\frac{2i}{\pi}\right)^2 \langle - | \hat{S}_x | - \rangle = \left(\frac{2i}{\pi}\right)^2 \cdot \frac{1}{2} = -\frac{2}{\pi^2}.
\end{align}

In [30]:
kets, coeffs = apply_Sz_projector_jax(psi[:, :nocc], 0., 1000)
kets = np.array(kets)
coeffs = np.array(coeffs)

In [31]:
# Projected spin average and overlap.
savg_tot = np.zeros(3, dtype=np.complex128)
ovlp_tot = 0.

for ibra, bra in enumerate(kets):
    for iket, ket in enumerate(kets):
        G, ovlp = get_greens(bra, ket)
        savg = spin_utils.get_spin_average_from_greens_ghf(G) * ovlp
        savg_tot += savg * coeffs[ibra].conj() * coeffs[iket]
        ovlp_tot += ovlp * coeffs[ibra].conj() * coeffs[iket]

In [32]:
print(f'<S> * ovlp = {savg_tot}')
print(f'ovlp = {ovlp_tot}')
print(f'\n<S> = {savg_tot / ovlp_tot}')

<S> * ovlp = [-0.20232+0.j -0.00064+0.j  0.     -0.j]
ovlp = (0.4052850679027681+0j)

<S> = [-0.49921+0.j -0.00157+0.j  0.     -0.j]


In [33]:
print(-2. / np.pi**2)

-0.20264236728467555


# Test with $|++\rangle = \frac{1}{2}(|\uparrow\uparrow\rangle + |\uparrow\downarrow\rangle + |\downarrow\uparrow\rangle + |\downarrow\downarrow\rangle)$

In [34]:
norb = 3
nocc = 2
psi = np.array([[1., 0., 0.], 
                [0., 1., 0.], 
                [0., 0., 1.], 
                [1., 0., 0.], 
                [0., 1., 0.], 
                [0., 0., 1.]]) / np.sqrt(2.)

In [35]:
# Initial spin average and overlap.
G, ovlp = get_greens(psi[:, :nocc], psi[:, :nocc])
savg = spin_utils.get_spin_average_from_greens_ghf(G) * ovlp

print(f'<S> = {savg}')
print(f'ovlp = {ovlp}')

<S> = [1. 0. 0.]
ovlp = 0.9999999999999996


## Project to $s_z = 1$

In [36]:
kets, coeffs = apply_Sz_projector_jax(psi[:, :nocc], 1., 10) # Should just give |uu>.
kets = np.array(kets)
coeffs = np.array(coeffs)

In [37]:
# Projected spin average and overlap.
savg_tot = np.zeros(3, dtype=np.complex128)
ovlp_tot = 0.

for ibra, bra in enumerate(kets):
    for iket, ket in enumerate(kets):
        G, ovlp = get_greens(bra, ket)
        savg = spin_utils.get_spin_average_from_greens_ghf(G) * ovlp
        savg_tot += savg * coeffs[ibra].conj() * coeffs[iket]
        ovlp_tot += ovlp * coeffs[ibra].conj() * coeffs[iket]

In [38]:
print(f'<S> * ovlp = {savg_tot}')
print(f'ovlp = {ovlp_tot}')
print(f'\n<S> = {savg_tot / ovlp_tot}')

<S> * ovlp = [-0.  -0.j -0.  -0.j  0.25+0.j]
ovlp = (0.2499999999999999+2.6020852139652106e-18j)

<S> = [-0.-0.j -0.-0.j  1.-0.j]


## Project to $s_z = -1$

In [39]:
kets, coeffs = apply_Sz_projector_jax(psi[:, :nocc], -1., 10) # Should just give |dd>.
kets = np.array(kets)
coeffs = np.array(coeffs)

In [40]:
# Projected spin average and overlap.
savg_tot = np.zeros(3, dtype=np.complex128)
ovlp_tot = 0.

for ibra, bra in enumerate(kets):
    for iket, ket in enumerate(kets):
        G, ovlp = get_greens(bra, ket)
        savg = spin_utils.get_spin_average_from_greens_ghf(G) * ovlp
        savg_tot += savg * coeffs[ibra].conj() * coeffs[iket]
        ovlp_tot += ovlp * coeffs[ibra].conj() * coeffs[iket]

In [41]:
print(f'<S> * ovlp = {savg_tot}')
print(f'ovlp = {ovlp_tot}')
print(f'\n<S> = {savg_tot / ovlp_tot}')

<S> * ovlp = [-0.  +0.j -0.  +0.j -0.25+0.j]
ovlp = (0.2499999999999999-2.6020852139652106e-18j)

<S> = [-0.+0.j -0.+0.j -1.-0.j]


## Project to $s_z = 0$

In [42]:
kets, coeffs = apply_Sz_projector_jax(psi[:, :nocc], 0., 10) # Should give |ud> + |du>.
kets = np.array(kets)
coeffs = np.array(coeffs)

In [43]:
# Projected spin average and overlap.
savg_tot = np.zeros(3, dtype=np.complex128)
ovlp_tot = 0.

for ibra, bra in enumerate(kets):
    for iket, ket in enumerate(kets):
        G, ovlp = get_greens(bra, ket)
        savg = spin_utils.get_spin_average_from_greens_ghf(G) * ovlp
        savg_tot += savg * coeffs[ibra].conj() * coeffs[iket]
        ovlp_tot += ovlp * coeffs[ibra].conj() * coeffs[iket]

In [44]:
print(f'<S> * ovlp = {savg_tot}')
print(f'ovlp = {ovlp_tot}')
print(f'\n<S> = {savg_tot / ovlp_tot}')

<S> * ovlp = [-0.+0.j -0.+0.j  0.-0.j]
ovlp = (0.5+0j)

<S> = [-0.+0.j -0.+0.j  0.-0.j]


# Test with $|+++\rangle$

In [45]:
norb = 3
nocc = 3
psi = np.array([[1., 0., 0.], 
                [0., 1., 0.], 
                [0., 0., 1.], 
                [1., 0., 0.], 
                [0., 1., 0.], 
                [0., 0., 1.]]) / np.sqrt(2.)

In [46]:
# Initial spin average and overlap.
G, ovlp = get_greens(psi[:, :nocc], psi[:, :nocc])
savg = spin_utils.get_spin_average_from_greens_ghf(G) * ovlp

print(f'<S> = {savg}')
print(f'ovlp = {ovlp}')

<S> = [1.5 0.  0. ]
ovlp = 0.9999999999999993


## Project to $s_z = 1.5$

In [47]:
kets, coeffs = apply_Sz_projector_jax(psi[:, :nocc], 1.5, 10) # Should just give |uuu>.
kets = np.array(kets)
coeffs = np.array(coeffs)

In [48]:
# Projected spin average and overlap.
savg_tot = np.zeros(3, dtype=np.complex128)
ovlp_tot = 0.

for ibra, bra in enumerate(kets):
    for iket, ket in enumerate(kets):
        G, ovlp = get_greens(bra, ket)
        savg = spin_utils.get_spin_average_from_greens_ghf(G) * ovlp
        savg_tot += savg * coeffs[ibra].conj() * coeffs[iket]
        ovlp_tot += ovlp * coeffs[ibra].conj() * coeffs[iket]

In [49]:
print(f'<S> * ovlp = {savg_tot}')
print(f'ovlp = {ovlp_tot}')
print(f'\n<S> = {savg_tot / ovlp_tot}')

<S> * ovlp = [-0.    +0.j -0.    -0.j  0.1875+0.j]
ovlp = (0.12499999999999996-8.673617379884035e-19j)

<S> = [-0. +0.j -0. -0.j  1.5+0.j]


## Project to $s_z = 0.5$

In [50]:
kets, coeffs = apply_Sz_projector_jax(psi[:, :nocc], 0.5, 10) # Should give |uud> + |udu> + |duu>.
kets = np.array(kets)
coeffs = np.array(coeffs)

In [51]:
# Projected spin average and overlap.
savg_tot = np.zeros(3, dtype=np.complex128)
ovlp_tot = 0.

for ibra, bra in enumerate(kets):
    for iket, ket in enumerate(kets):
        G, ovlp = get_greens(bra, ket)
        savg = spin_utils.get_spin_average_from_greens_ghf(G) * ovlp
        savg_tot += savg * coeffs[ibra].conj() * coeffs[iket]
        ovlp_tot += ovlp * coeffs[ibra].conj() * coeffs[iket]

In [52]:
print(f'<S> * ovlp = {savg_tot}')
print(f'ovlp = {ovlp_tot}')
print(f'\n<S> = {savg_tot / ovlp_tot}')

<S> * ovlp = [-0.    -0.j -0.    -0.j  0.1875-0.j]
ovlp = (0.3749999999999997-1.734723475976807e-18j)

<S> = [-0. -0.j -0. -0.j  0.5-0.j]


## Project to $s_z = -0.5$

In [53]:
kets, coeffs = apply_Sz_projector_jax(psi[:, :nocc], -0.5, 10) # Should give |ddu> + |dud> + |udd>.
kets = np.array(kets)
coeffs = np.array(coeffs)

In [54]:
# Projected spin average and overlap.
savg_tot = np.zeros(3, dtype=np.complex128)
ovlp_tot = 0.

for ibra, bra in enumerate(kets):
    for iket, ket in enumerate(kets):
        G, ovlp = get_greens(bra, ket)
        savg = spin_utils.get_spin_average_from_greens_ghf(G) * ovlp
        savg_tot += savg * coeffs[ibra].conj() * coeffs[iket]
        ovlp_tot += ovlp * coeffs[ibra].conj() * coeffs[iket]

In [55]:
print(f'<S> * ovlp = {savg_tot}')
print(f'ovlp = {ovlp_tot}')
print(f'\n<S> = {savg_tot / ovlp_tot}')

<S> * ovlp = [-0.    +0.j -0.    +0.j -0.1875-0.j]
ovlp = (0.3749999999999997+1.734723475976807e-18j)

<S> = [-0. +0.j -0. +0.j -0.5-0.j]


## Project to $s_z = -1.5$

In [56]:
kets, coeffs = apply_Sz_projector_jax(psi[:, :nocc], -1.5, 10) # Should just give |ddd>.
kets = np.array(kets)
coeffs = np.array(coeffs)

In [57]:
# Projected spin average and overlap.
savg_tot = np.zeros(3, dtype=np.complex128)
ovlp_tot = 0.

for ibra, bra in enumerate(kets):
    for iket, ket in enumerate(kets):
        G, ovlp = get_greens(bra, ket)
        savg = spin_utils.get_spin_average_from_greens_ghf(G) * ovlp
        savg_tot += savg * coeffs[ibra].conj() * coeffs[iket]
        ovlp_tot += ovlp * coeffs[ibra].conj() * coeffs[iket]

In [58]:
print(f'<S> * ovlp = {savg_tot}')
print(f'ovlp = {ovlp_tot}')
print(f'\n<S> = {savg_tot / ovlp_tot}')

<S> * ovlp = [-0.    -0.j -0.    +0.j -0.1875+0.j]
ovlp = (0.12499999999999996+8.673617379884035e-19j)

<S> = [-0. -0.j -0. +0.j -1.5+0.j]


# Test `get_real_wavefunction`

In [43]:
ngrid = 5
norb = 3
nocc = 3
kets = np.random.random((ngrid, 2*norb, nocc)) + 1.j * np.random.random((ngrid, 2*norb, nocc))
coeffs = np.random.random(ngrid) + 1.j * np.random.random(ngrid)

for ig in range(ngrid):
    for icol in range(nocc):
        kets[ig, :, icol] /= sp.linalg.norm(kets[ig, :, icol])

In [44]:
real_kets, real_coeffs = get_real_wavefunction(kets, coeffs)
sum_kets = np.sum(kets * coeffs[:, None, None], axis=0)
sum_real_kets = np.sum(real_kets * real_coeffs[:, None, None], axis=0)

print(f'\nsum_kets = \n{sum_kets}')
print(f'\nsum_real_kets = \n{sum_real_kets}')

np.testing.assert_allclose(np.amax(np.absolute(sum_real_kets.imag)), 0., atol=1e-15)


sum_kets = 
[[-0.24156+1.11192j -0.76672+1.28439j -0.41995+1.20624j]
 [-0.41842+1.45514j -0.25815+1.32491j -0.91681+1.4229j ]
 [-0.3939 +1.20771j -0.07863+1.10225j -0.27349+1.06581j]
 [-0.66283+0.91757j -0.30296+1.00648j -0.29685+1.03099j]
 [-0.70909+1.21201j -0.52768+1.02683j -0.25307+1.54514j]
 [-0.65027+1.13509j -0.49659+1.23317j -0.12859+0.71625j]]

sum_real_kets = 
[[-0.34161+0.j -1.08431+0.j -0.59389+0.j]
 [-0.59174-0.j -0.36508+0.j -1.29656+0.j]
 [-0.55706-0.j -0.1112 -0.j -0.38678-0.j]
 [-0.93738+0.j -0.42845-0.j -0.41982-0.j]
 [-1.00281+0.j -0.74626+0.j -0.35789-0.j]
 [-0.91962+0.j -0.70228-0.j -0.18185-0.j]]


# Test `get_energy`

In [59]:
U = 4
nx = 2
ny = 2
nup = 2
ndown = 2
open_x = False
verbose = 3

# -----------------------------------------------------------------------------
# Settings.
lattice = lattices.triangular_grid(nx, ny, open_x=open_x)
n_sites = lattice.n_sites
n_elec = (nup, ndown)
nocc = sum(n_elec)
filling = nocc / (2*n_sites)
if verbose: print(f'\n# Filling factor = {filling}')


# Filling factor = 0.5


In [60]:
integrals = {}
integrals["h0"] = 0.0

h1 = -1.0 * lattice.create_adjacency_matrix()
integrals["h1"] = h1

h2 = np.zeros((n_sites, n_sites, n_sites, n_sites))
for i in range(n_sites):
    h2[i, i, i, i] = U
integrals["h2"] = ao2mo.restore(8, h2, n_sites)

In [61]:
# Get Choleskies.
chol_cut = 1e-10
eri = ao2mo.restore(4, integrals["h2"], n_sites)
chol0 = pyscf_interface.modified_cholesky(eri, max_error=chol_cut)
nchol = chol0.shape[0]
chol = np.zeros((nchol, n_sites, n_sites))

for i in range(nchol):
    for m in range(n_sites):
        for n in range(m + 1):
            triind = m * (m + 1) // 2 + n
            chol[i, m, n] = chol0[i, triind]
            chol[i, n, m] = chol0[i, triind]

chol = chol.reshape(nchol, n_sites**2)
_eri = (chol.T @ chol).reshape((n_sites,) * 4)
max_absdiff = np.amax(np.absolute(h2 - _eri))
print(f'max_absdiff = {max_absdiff}')

max_absdiff = 5.000000413701855e-11


In [62]:
# make dummy molecule
mol = gto.Mole()
mol.nelectron = nocc
mol.incore_anyway = True
mol.spin = abs(n_elec[0] - n_elec[1])
mol.verbose = 3
mol.build()

# GHF.
gmf = scf.GHF(mol)
gmf.get_hcore = lambda *args: sp.linalg.block_diag(integrals["h1"], integrals["h1"])
gmf.get_ovlp = lambda *args: np.eye(2 * n_sites)
gmf._eri = ao2mo.restore(8, integrals["h2"], n_sites)

seed = 262
np.random.seed(seed)
dm_init = np.random.random((2*n_sites, 2*n_sites))
dm_init += dm_init.T.conj()

gmf.kernel(dm_init)
mo1 = gmf.stability(external=True)
gmf = gmf.newton().run(mo1, gmf.mo_occ)
mo1 = gmf.stability(external=True)
gmf = gmf.newton().run(mo1, gmf.mo_occ)
mo1 = gmf.stability(external=True)
gmf = gmf.newton().run(mo1, gmf.mo_occ)
mo1 = gmf.stability(external=True)

ghf_coeff = gmf.mo_coeff
ghf_rdm1 = gmf.make_rdm1()
ao_ovlp = np.eye(n_sites)
epsilon0, mu, spin_axis = spin_utils.spin_collinearity_test(ghf_coeff[:, :nocc], ao_ovlp, verbose=verbose)

converged SCF energy = -1.58788806326219  <S^2> = 2.3595938  2S+1 = 3.2308474
<class 'pyscf.scf.ghf.GHF'> wavefunction has an internal instability


Overwritten attributes  get_ovlp get_hcore  of <class 'pyscf.scf.ghf.GHF'>
Overwritten attributes  get_ovlp get_hcore  of <class 'pyscf.soscf.newton_ah.SecondOrderGHF'>


converged SCF energy = -1.76329782852721  <S^2> = 1.5301269  2S+1 = 2.6684279
<class 'pyscf.soscf.newton_ah.SecondOrderGHF'> wavefunction is stable in the internal stability analysis
converged SCF energy = -1.76329782855061  <S^2> = 1.530127  2S+1 = 2.668428
<class 'pyscf.soscf.newton_ah.SecondOrderGHF'> wavefunction is stable in the internal stability analysis
converged SCF energy = -1.76329782855372  <S^2> = 1.530127  2S+1 = 2.668428
<class 'pyscf.soscf.newton_ah.SecondOrderGHF'> wavefunction is stable in the internal stability analysis

# ----------------------
# Spin collinearity test
# ----------------------
# epsilon0 = 5.600759033189529e-07
# non integer value indicates non-collinearity

# minimum mu = 0.2882556229567591
# Value is 0 iff wavefunction is collinear

# If mu = 0, the collinear spin axis is: 
[-0.31794-0.j -0.     +0.j -0.94811+0.j]


In [63]:
psi = ghf_coeff[:, :nocc]
psiTconj = psi.T.conj()
rotchol = np.zeros((2, nchol, nocc, n_sites), dtype=np.complex128)

for i in range(nchol):
    rotchol[0, i] = psiTconj[:, :n_sites] @ chol[i].reshape((n_sites, n_sites))
    rotchol[1, i] = psiTconj[:, n_sites:] @ chol[i].reshape((n_sites, n_sites))

energy = get_energy(psi, psi, [h1, h1], rotchol, 0.)
energy = get_energy_jax(psi, psi, [h1, h1], rotchol, 0.)
np.testing.assert_allclose(energy, gmf.e_tot)

In [64]:
def get_energy_with_eri(bra, ket, h1, eri, enuc):
    norb = h1.shape[0]
    nocc = bra.shape[1]
    G, ovlp = get_greens(bra, ket)
    Gaa = G[:norb, :norb]
    Gab = G[:norb, norb:]
    Gba = G[norb:, :norb]
    Gbb = G[norb:, norb:]
    Gcharge = Gaa + Gbb
    energy = enuc
    energy += np.trace(h1 @ Gcharge)
    energy += 0.5 * np.einsum('ijkl,lk,ji->', eri, Gcharge, Gcharge) # Ej
    energy -= 0.5 * np.einsum('ijkl,jk,li->', eri, Gaa, Gaa) # Ek
    energy -= 0.5 * np.einsum('ijkl,jk,li->', eri, Gbb, Gbb)
    energy -= 0.5 * np.einsum('ijkl,jk,li->', eri, Gab, Gba)
    energy -= 0.5 * np.einsum('ijkl,jk,li->', eri, Gba, Gab)
    return energy

In [65]:
m = 0.5 * (nup - ndown)
kets, coeffs = apply_Sz_projector_jax(psi, m, 10)
kets = np.array(kets)
coeffs = np.array(coeffs)
kets.shape

(10, 8, 4)

In [66]:
for bra in kets:
    for ket in kets:
        energy_test = get_energy(bra, ket, [h1, h1], rotchol, 0.)
        energy_ref = get_energy_with_eri(bra, ket, h1, h2, 0.)
        np.testing.assert_allclose(energy_test, energy_ref, atol=1e-15)

# Test `get_projected_energy`

## Test 1: UHF state with $s_z = 0$

In [67]:
U = 4
nx = 2
ny = 2
nup = 2
ndown = 2
open_x = False
verbose = 3

# -----------------------------------------------------------------------------
# Settings.
lattice = lattices.triangular_grid(nx, ny, open_x=open_x)
n_sites = lattice.n_sites
n_elec = (nup, ndown)
nocc = sum(n_elec)
filling = nocc / (2*n_sites)
if verbose: print(f'\n# Filling factor = {filling}')


# Filling factor = 0.5


In [68]:
integrals = {}
integrals["h0"] = 0.0

h1 = -1.0 * lattice.create_adjacency_matrix()
integrals["h1"] = h1

h2 = np.zeros((n_sites, n_sites, n_sites, n_sites))
for i in range(n_sites):
    h2[i, i, i, i] = U
integrals["h2"] = ao2mo.restore(8, h2, n_sites)

In [69]:
# Get Choleskies.
chol_cut = 1e-10
eri = ao2mo.restore(4, integrals["h2"], n_sites)
chol0 = pyscf_interface.modified_cholesky(eri, max_error=chol_cut)
nchol = chol0.shape[0]
chol = np.zeros((nchol, n_sites, n_sites))

for i in range(nchol):
    for m in range(n_sites):
        for n in range(m + 1):
            triind = m * (m + 1) // 2 + n
            chol[i, m, n] = chol0[i, triind]
            chol[i, n, m] = chol0[i, triind]

chol = chol.reshape(nchol, n_sites**2)
_eri = (chol.T @ chol).reshape((n_sites,) * 4)
max_absdiff = np.amax(np.absolute(h2 - _eri))
print(f'max_absdiff = {max_absdiff}')

max_absdiff = 5.000000413701855e-11


In [70]:
# make dummy molecule
mol = gto.Mole()
mol.nelectron = nocc
mol.incore_anyway = True
mol.spin = abs(n_elec[0] - n_elec[1])
mol.verbose = 3
mol.build()

# UHF.
umf = scf.UHF(mol)
umf.get_hcore = lambda *args: integrals["h1"]
umf.get_ovlp = lambda *args: np.eye(n_sites)
umf._eri = ao2mo.restore(8, integrals["h2"], n_sites)

seed = 262
np.random.seed(seed)
dm_init = np.random.random((2, n_sites, n_sites))
dm_init += dm_init.transpose(0, 2, 1).conj()

umf.kernel(dm_init)
mo1 = umf.stability(external=True)
umf = umf.newton().run(mo1[0], umf.mo_occ)
mo1 = umf.stability(external=True)
umf = umf.newton().run(mo1[0], umf.mo_occ)
mo1 = umf.stability(external=True)
umf = umf.newton().run(mo1[0], umf.mo_occ)
mo1 = umf.stability(external=True)

uhf_coeff = umf.mo_coeff
uhf_rdm1 = umf.make_rdm1()

psi = np.zeros((2*n_sites, nocc))
psi[:n_sites, :nup] = uhf_coeff[0, :, umf.mo_occ[0] > 0].T
psi[n_sites:, nup:] = uhf_coeff[1, :, umf.mo_occ[1] > 0].T

ao_ovlp = np.eye(n_sites)
epsilon0, mu, spin_axis = spin_utils.spin_collinearity_test(psi, ao_ovlp, verbose=verbose)

converged SCF energy = -1.76329782855449  <S^2> = 1.399794  2S+1 = 2.5688861
<class 'pyscf.scf.uhf.UHF'> wavefunction is stable in the internal stability analysis
<class 'pyscf.scf.uhf.UHF'> wavefunction is stable in the real -> complex stability analysis
<class 'pyscf.scf.uhf.UHF'> wavefunction is stable in the UHF/UKS -> GHF/GKS stability analysis
converged SCF energy = -1.76329782855458  <S^2> = 1.3997941  2S+1 = 2.5688862
<class 'pyscf.soscf.newton_ah.SecondOrderUHF'> wavefunction is stable in the internal stability analysis

WARN: Not enough eigenvectors (len(x0)=1, nroots=3)

<class 'pyscf.soscf.newton_ah.SecondOrderUHF'> wavefunction is stable in the real -> complex stability analysis
<class 'pyscf.soscf.newton_ah.SecondOrderUHF'> wavefunction is stable in the UHF/UKS -> GHF/GKS stability analysis
converged SCF energy = -1.76329782855459  <S^2> = 1.3997941  2S+1 = 2.5688862
<class 'pyscf.soscf.newton_ah.SecondOrderUHF'> wavefunction is stable in the internal stability analysis



Overwritten attributes  get_ovlp get_hcore  of <class 'pyscf.scf.uhf.UHF'>
Overwritten attributes  get_ovlp get_hcore  of <class 'pyscf.soscf.newton_ah.SecondOrderUHF'>


In [73]:
m = 0.5 * (nup - ndown)
kets, coeffs = apply_Sz_projector_jax(psi, m, 10)
kets = np.array(kets)
coeffs = np.array(coeffs)

In [74]:
# Projected spin average and overlap.
savg_tot = np.zeros(3, dtype=np.complex128)
ovlp_tot = 0.

for ibra, bra in enumerate(kets):
    for iket, ket in enumerate(kets):
        G, ovlp = get_greens(bra, ket)
        savg = spin_utils.get_spin_average_from_greens_ghf(G) * ovlp
        savg_tot += savg * coeffs[ibra].conj() * coeffs[iket]
        ovlp_tot += ovlp * coeffs[ibra].conj() * coeffs[iket]

In [75]:
print(f'<S> * ovlp = {savg_tot}')
print(f'ovlp = {ovlp_tot}')
print(f'\n<S> = {savg_tot / ovlp_tot}')

<S> * ovlp = [0.+0.j 0.+0.j 0.-0.j]
ovlp = (1.0000000000000007+3.851859888774472e-34j)

<S> = [0.+0.j 0.+0.j 0.-0.j]


In [77]:
e_proj = get_projected_energy(psi, m, [h1, h1], chol, 0., 10)
e_proj = get_projected_energy_jax(psi, m, [h1, h1], chol, 0., 10)
np.testing.assert_allclose(e_proj, umf.e_tot)

## Test 1: UHF state with $s_z = 1.5$

In [78]:
U = 4
nx = 2
ny = 2
nup = 4
ndown = 1
open_x = False
verbose = 3

# -----------------------------------------------------------------------------
# Settings.
lattice = lattices.triangular_grid(nx, ny, open_x=open_x)
n_sites = lattice.n_sites
n_elec = (nup, ndown)
nocc = sum(n_elec)
filling = nocc / (2*n_sites)
if verbose: print(f'\n# Filling factor = {filling}')


# Filling factor = 0.625


In [79]:
integrals = {}
integrals["h0"] = 0.0

h1 = -1.0 * lattice.create_adjacency_matrix()
integrals["h1"] = h1

h2 = np.zeros((n_sites, n_sites, n_sites, n_sites))
for i in range(n_sites):
    h2[i, i, i, i] = U
integrals["h2"] = ao2mo.restore(8, h2, n_sites)

In [80]:
# Get Choleskies.
chol_cut = 1e-10
eri = ao2mo.restore(4, integrals["h2"], n_sites)
chol0 = pyscf_interface.modified_cholesky(eri, max_error=chol_cut)
nchol = chol0.shape[0]
chol = np.zeros((nchol, n_sites, n_sites))

for i in range(nchol):
    for m in range(n_sites):
        for n in range(m + 1):
            triind = m * (m + 1) // 2 + n
            chol[i, m, n] = chol0[i, triind]
            chol[i, n, m] = chol0[i, triind]

chol = chol.reshape(nchol, n_sites**2)
_eri = (chol.T @ chol).reshape((n_sites,) * 4)
max_absdiff = np.amax(np.absolute(h2 - _eri))
print(f'max_absdiff = {max_absdiff}')

max_absdiff = 5.000000413701855e-11


In [81]:
# make dummy molecule
mol = gto.Mole()
mol.nelectron = nocc
mol.incore_anyway = True
mol.spin = abs(n_elec[0] - n_elec[1])
mol.verbose = 3
mol.build()

# UHF.
umf = scf.UHF(mol)
umf.get_hcore = lambda *args: integrals["h1"]
umf.get_ovlp = lambda *args: np.eye(n_sites)
umf._eri = ao2mo.restore(8, integrals["h2"], n_sites)

seed = 262
np.random.seed(seed)
dm_init = np.random.random((2, n_sites, n_sites))
dm_init += dm_init.transpose(0, 2, 1).conj()

umf.kernel(dm_init)
mo1 = umf.stability(external=True)
umf = umf.newton().run(mo1[0], umf.mo_occ)
mo1 = umf.stability(external=True)
umf = umf.newton().run(mo1[0], umf.mo_occ)
mo1 = umf.stability(external=True)
umf = umf.newton().run(mo1[0], umf.mo_occ)
mo1 = umf.stability(external=True)

uhf_coeff = umf.mo_coeff
uhf_rdm1 = umf.make_rdm1()

psi = np.zeros((2*n_sites, nocc))
psi[:n_sites, :nup] = uhf_coeff[0, :, umf.mo_occ[0] > 0].T
psi[n_sites:, nup:] = uhf_coeff[1, :, umf.mo_occ[1] > 0].T

ao_ovlp = np.eye(n_sites)
epsilon0, mu, spin_axis = spin_utils.spin_collinearity_test(psi, ao_ovlp, verbose=verbose)

converged SCF energy = 0.999999999999997  <S^2> = 3.75  2S+1 = 4

WARN: Not enough eigenvectors (len(x0)=1, nroots=3)

<class 'pyscf.scf.uhf.UHF'> wavefunction is stable in the internal stability analysis

WARN: Not enough eigenvectors (len(x0)=1, nroots=3)

<class 'pyscf.scf.uhf.UHF'> wavefunction is stable in the real -> complex stability analysis
<class 'pyscf.scf.uhf.UHF'> wavefunction is stable in the UHF/UKS -> GHF/GKS stability analysis
converged SCF energy = 0.999999999999997  <S^2> = 3.75  2S+1 = 4

WARN: Not enough eigenvectors (len(x0)=1, nroots=3)

<class 'pyscf.soscf.newton_ah.SecondOrderUHF'> wavefunction is stable in the internal stability analysis

WARN: Not enough eigenvectors (len(x0)=1, nroots=3)

<class 'pyscf.soscf.newton_ah.SecondOrderUHF'> wavefunction is stable in the real -> complex stability analysis
<class 'pyscf.soscf.newton_ah.SecondOrderUHF'> wavefunction is stable in the UHF/UKS -> GHF/GKS stability analysis
converged SCF energy = 0.999999999999996  <S^2>

In [82]:
m = 0.5 * (nup - ndown)
kets, coeffs = apply_Sz_projector_jax(psi, m, 10)
kets = np.array(kets)
coeffs = np.array(coeffs)

In [83]:
# Projected spin average and overlap.
savg_tot = np.zeros(3, dtype=np.complex128)
ovlp_tot = 0.

for ibra, bra in enumerate(kets):
    for iket, ket in enumerate(kets):
        G, ovlp = get_greens(bra, ket)
        savg = spin_utils.get_spin_average_from_greens_ghf(G) * ovlp
        savg_tot += savg * coeffs[ibra].conj() * coeffs[iket]
        ovlp_tot += ovlp * coeffs[ibra].conj() * coeffs[iket]

In [84]:
print(f'<S> * ovlp = {savg_tot}')
print(f'ovlp = {ovlp_tot}')
print(f'\n<S> = {savg_tot / ovlp_tot}')

<S> * ovlp = [0. +0.j 0. +0.j 1.5+0.j]
ovlp = (0.9999999999999982-3.25862638659177e-18j)

<S> = [0. +0.j 0. +0.j 1.5+0.j]


In [86]:
e_proj = get_projected_energy(psi, m, [h1, h1], chol, 0., 10)
e_proj = get_projected_energy_jax(psi, m, [h1, h1], chol, 0., 10)
np.testing.assert_allclose(e_proj, umf.e_tot)

# Test 3: Check with FCI

In [471]:
U = 4
nx = 2
ny = 2
nup = 2
ndown = 2
open_x = False
verbose = 3

# -----------------------------------------------------------------------------
# Settings.
lattice = lattices.triangular_grid(nx, ny, open_x=open_x)
n_sites = lattice.n_sites
n_elec = (nup, ndown)
nocc = sum(n_elec)
filling = nocc / (2*n_sites)
if verbose: print(f'\n# Filling factor = {filling}')


# Filling factor = 0.5


In [472]:
integrals = {}
integrals["h0"] = 0.0

h1 = -1.0 * lattice.create_adjacency_matrix()
integrals["h1"] = h1

h2 = np.zeros((n_sites, n_sites, n_sites, n_sites))
for i in range(n_sites):
    h2[i, i, i, i] = U
integrals["h2"] = ao2mo.restore(8, h2, n_sites)

In [473]:
# Get Choleskies.
chol_cut = 1e-10
eri = ao2mo.restore(4, integrals["h2"], n_sites)
chol0 = pyscf_interface.modified_cholesky(eri, max_error=chol_cut)
nchol = chol0.shape[0]
chol = np.zeros((nchol, n_sites, n_sites))

for i in range(nchol):
    for m in range(n_sites):
        for n in range(m + 1):
            triind = m * (m + 1) // 2 + n
            chol[i, m, n] = chol0[i, triind]
            chol[i, n, m] = chol0[i, triind]

chol = chol.reshape(nchol, n_sites**2)
_eri = (chol.T @ chol).reshape((n_sites,) * 4)
max_absdiff = np.amax(np.absolute(h2 - _eri))
print(f'max_absdiff = {max_absdiff}')

max_absdiff = 5.000000413701855e-11


In [474]:
# make dummy molecule
mol = gto.Mole()
mol.nelectron = nocc
mol.incore_anyway = True
mol.spin = abs(n_elec[0] - n_elec[1])
mol.verbose = 3
mol.build()

# UHF.
umf = scf.UHF(mol)
umf.get_hcore = lambda *args: integrals["h1"]
umf.get_ovlp = lambda *args: np.eye(n_sites)
umf._eri = ao2mo.restore(8, integrals["h2"], n_sites)

seed = 262
np.random.seed(seed)
dm_init = np.random.random((2, n_sites, n_sites))
dm_init += dm_init.transpose(0, 2, 1).conj()

umf.kernel(dm_init)
mo1 = umf.stability(external=True)
umf = umf.newton().run(mo1[0], umf.mo_occ)
mo1 = umf.stability(external=True)
umf = umf.newton().run(mo1[0], umf.mo_occ)
mo1 = umf.stability(external=True)
umf = umf.newton().run(mo1[0], umf.mo_occ)
mo1 = umf.stability(external=True)

uhf_coeff = umf.mo_coeff
uhf_rdm1 = umf.make_rdm1()

psi = np.zeros((2*n_sites, nocc))
psi[:n_sites, :nup] = uhf_coeff[0, :, umf.mo_occ[0] > 0].T
psi[n_sites:, nup:] = uhf_coeff[1, :, umf.mo_occ[1] > 0].T

ao_ovlp = np.eye(n_sites)
epsilon0, mu, spin_axis = spin_utils.spin_collinearity_test(psi, ao_ovlp, verbose=verbose)

converged SCF energy = -1.76329782855449  <S^2> = 1.399794  2S+1 = 2.5688861
<class 'pyscf.scf.uhf.UHF'> wavefunction is stable in the internal stability analysis
<class 'pyscf.scf.uhf.UHF'> wavefunction is stable in the real -> complex stability analysis
<class 'pyscf.scf.uhf.UHF'> wavefunction is stable in the UHF/UKS -> GHF/GKS stability analysis
converged SCF energy = -1.76329782855458  <S^2> = 1.3997941  2S+1 = 2.5688862
<class 'pyscf.soscf.newton_ah.SecondOrderUHF'> wavefunction is stable in the internal stability analysis

WARN: Not enough eigenvectors (len(x0)=1, nroots=3)

<class 'pyscf.soscf.newton_ah.SecondOrderUHF'> wavefunction is stable in the real -> complex stability analysis
<class 'pyscf.soscf.newton_ah.SecondOrderUHF'> wavefunction is stable in the UHF/UKS -> GHF/GKS stability analysis
converged SCF energy = -1.76329782855459  <S^2> = 1.3997941  2S+1 = 2.5688862
<class 'pyscf.soscf.newton_ah.SecondOrderUHF'> wavefunction is stable in the internal stability analysis



In [475]:
from pyscf.fci import cistring

alpha_strs = list(cistring.gen_occslst(range(n_sites), nup))
beta_strs = list(cistring.gen_occslst(range(n_sites), ndown))

print(alpha_strs)
print(beta_strs)

[OIndexList([0, 1], dtype=int32), OIndexList([0, 2], dtype=int32), OIndexList([1, 2], dtype=int32), OIndexList([0, 3], dtype=int32), OIndexList([1, 3], dtype=int32), OIndexList([2, 3], dtype=int32)]
[OIndexList([0, 1], dtype=int32), OIndexList([0, 2], dtype=int32), OIndexList([1, 2], dtype=int32), OIndexList([0, 3], dtype=int32), OIndexList([1, 3], dtype=int32), OIndexList([2, 3], dtype=int32)]


In [476]:
slater_dets = [(a_str, b_str) for a_str in alpha_strs for b_str in beta_strs]
slater_dets

[(OIndexList([0, 1], dtype=int32), OIndexList([0, 1], dtype=int32)),
 (OIndexList([0, 1], dtype=int32), OIndexList([0, 2], dtype=int32)),
 (OIndexList([0, 1], dtype=int32), OIndexList([1, 2], dtype=int32)),
 (OIndexList([0, 1], dtype=int32), OIndexList([0, 3], dtype=int32)),
 (OIndexList([0, 1], dtype=int32), OIndexList([1, 3], dtype=int32)),
 (OIndexList([0, 1], dtype=int32), OIndexList([2, 3], dtype=int32)),
 (OIndexList([0, 2], dtype=int32), OIndexList([0, 1], dtype=int32)),
 (OIndexList([0, 2], dtype=int32), OIndexList([0, 2], dtype=int32)),
 (OIndexList([0, 2], dtype=int32), OIndexList([1, 2], dtype=int32)),
 (OIndexList([0, 2], dtype=int32), OIndexList([0, 3], dtype=int32)),
 (OIndexList([0, 2], dtype=int32), OIndexList([1, 3], dtype=int32)),
 (OIndexList([0, 2], dtype=int32), OIndexList([2, 3], dtype=int32)),
 (OIndexList([1, 2], dtype=int32), OIndexList([0, 1], dtype=int32)),
 (OIndexList([1, 2], dtype=int32), OIndexList([0, 2], dtype=int32)),
 (OIndexList([1, 2], dtype=int32),

In [477]:
umf.mo_coeff

array([[[-0.30319, -0.     , -0.63881,  0.70711],
        [-0.30319, -0.     , -0.63881, -0.70711],
        [-0.63881, -0.70711,  0.30319,  0.     ],
        [-0.63881,  0.70711,  0.30319,  0.     ]],

       [[-0.63881,  0.70711,  0.30319, -0.     ],
        [-0.63881, -0.70711,  0.30319, -0.     ],
        [-0.30319, -0.     , -0.63881, -0.70711],
        [-0.30319, -0.     , -0.63881,  0.70711]]])

In [478]:
umf.mo_energy

array([[-1.58155,  1.36771,  3.58155,  4.63229],
       [-1.58155,  1.36771,  3.58155,  4.63229]])

In [480]:
mol.verbose = 5
ci = fci.FCI(mol)
ci = fci.addons.fix_spin_(ci, ss=0)
e, ci_coeffs = ci.kernel(
    h1e=integrals["h1"], eri=integrals["h2"], norb=n_sites, nelec=n_elec, max_cycle=500
)
print(e)

davidson 0 1  |r|= 2.84  e= [0.2]  max|de|=  0.2  lindep=    1
davidson 1 2  |r|= 1.51  e= [-0.45819]  max|de|= -0.658  lindep= 0.705
davidson 2 3  |r|= 1.06  e= [-1.7044]  max|de|= -1.25  lindep= 0.999
davidson 3 4  |r|= 0.351  e= [-1.87417]  max|de|= -0.17  lindep= 0.989
davidson 4 5  |r|= 0.477  e= [-1.92717]  max|de|= -0.053  lindep= 0.985
davidson 5 6  |r|= 0.0478  e= [-2.10249]  max|de|= -0.175  lindep= 0.883
davidson 6 7  |r|= 2.28e-05  e= [-2.10275]  max|de|= -0.000254  lindep=    1
root 0 converged  |r|= 7.28e-06  e= -2.102748483446554  max|de|= -5.8e-11
converged 7 8  |r|= 7.28e-06  e= [-2.10275]  max|de|= -5.8e-11
-2.102748483446554


In [481]:
ci_coeffs

FCIvector([[-0.05109, -0.     ,  0.12905, -0.12905,  0.     , -0.24548],
           [-0.     ,  0.05109,  0.12905,  0.12904,  0.24548, -0.     ],
           [ 0.12905,  0.12905,  0.     ,  0.49096,  0.12904,  0.12904],
           [-0.12905,  0.12904,  0.49096,  0.     ,  0.12905, -0.12904],
           [ 0.     ,  0.24548,  0.12904,  0.12905,  0.05109,  0.     ],
           [-0.24548, -0.     ,  0.12904, -0.12904,  0.     , -0.05109]])

In [482]:
len(slater_dets)

36

In [432]:
umf.e_tot

0.9999999999999947

In [421]:
e_proj

(0.9999999999625035-1.3877787807294141e-17j)

In [464]:
occslst = fci.cistring.gen_occslst(range(n_sites), ndown)
occslst

OIndexList([[0],
            [1],
            [2],
            [3]], dtype=int32)

In [452]:
occslst

OIndexList([[0, 1],
            [0, 2],
            [1, 2],
            [0, 3],
            [1, 3],
            [2, 3]], dtype=int32)

## Using a 2-site Ising model as example

In [267]:
norb = 2
nocc = 2
psi = 1./np.sqrt(2.) * np.array([[1., 0.],  # (site 1, up)
                                 [0., 1.],  # (site 2, up) 
                                 [1., 0.],  # (site 1, down)
                                 [0., 1.]]) # (site 2, down)

In [269]:
h1 = np.zeros((2*norb, 2*norb))
eri = np.zeros((2*norb, 2*norb, 2*norb, 2*norb))

for i in range(2*norb):
    spin_i = i // 2
    site_i = i % 2
    
    for j in range(2*norb):
        spin_j = j // 2
        site_j = j % 2
        
        if spin_i == spin_j: eri[i, i, j, j] = 1.
        if spin_i != spin_j: eri[i, i, j, j] = -1.

H = np.array([[0., 1., 0., -1.],
              [1., 0., -1., 0.],
              [0., -1., 0., 1.],
              [-1., 0., 1., 0.]])

In [264]:
get_projected_energy(psi, 0., [h1, h1], chol, 0., 10)

(-22.25000000001807-1.848290669299883e-15j)

# Test `gradient_descent`

In [89]:
U = 4
nx = 4
ny = 4
nup = 3
ndown = 3
open_x = False
verbose = 3

# -----------------------------------------------------------------------------
# Settings.
lattice = lattices.triangular_grid(nx, ny, open_x=open_x)
n_sites = lattice.n_sites
n_elec = (nup, ndown)
nocc = sum(n_elec)
filling = nocc / (2*n_sites)
if verbose: print(f'\n# Filling factor = {filling}')


# Filling factor = 0.1875


In [90]:
integrals = {}
integrals["h0"] = 0.0

h1 = -1.0 * lattice.create_adjacency_matrix()
integrals["h1"] = h1

h2 = np.zeros((n_sites, n_sites, n_sites, n_sites))
for i in range(n_sites):
    h2[i, i, i, i] = U
integrals["h2"] = ao2mo.restore(8, h2, n_sites)

In [91]:
# Get Choleskies.
chol_cut = 1e-10
eri = ao2mo.restore(4, integrals["h2"], n_sites)
chol0 = pyscf_interface.modified_cholesky(eri, max_error=chol_cut)
nchol = chol0.shape[0]
chol = np.zeros((nchol, n_sites, n_sites))

for i in range(nchol):
    for m in range(n_sites):
        for n in range(m + 1):
            triind = m * (m + 1) // 2 + n
            chol[i, m, n] = chol0[i, triind]
            chol[i, n, m] = chol0[i, triind]

chol = chol.reshape(nchol, n_sites**2)
_eri = (chol.T @ chol).reshape((n_sites,) * 4)
max_absdiff = np.amax(np.absolute(h2 - _eri))
print(f'max_absdiff = {max_absdiff}')

max_absdiff = 5.000000413701855e-11


In [92]:
# make dummy molecule
mol = gto.Mole()
mol.nelectron = nocc
mol.incore_anyway = True
mol.spin = abs(n_elec[0] - n_elec[1])
mol.verbose = 3
mol.build()

# GHF.
gmf = scf.GHF(mol)
gmf.get_hcore = lambda *args: sp.linalg.block_diag(integrals["h1"], integrals["h1"])
gmf.get_ovlp = lambda *args: np.eye(2 * n_sites)
gmf._eri = ao2mo.restore(8, integrals["h2"], n_sites)

seed = 262
np.random.seed(seed)
dm_init = np.random.random((2*n_sites, 2*n_sites))
dm_init += dm_init.T.conj()

gmf.kernel(dm_init)
mo1 = gmf.stability(external=True)
gmf = gmf.newton().run(mo1, gmf.mo_occ)
mo1 = gmf.stability(external=True)
gmf = gmf.newton().run(mo1, gmf.mo_occ)
mo1 = gmf.stability(external=True)
gmf = gmf.newton().run(mo1, gmf.mo_occ)
mo1 = gmf.stability(external=True)

ghf_coeff = gmf.mo_coeff
ghf_rdm1 = gmf.make_rdm1()
ao_ovlp = np.eye(n_sites)
epsilon0, mu, spin_axis = spin_utils.spin_collinearity_test(ghf_coeff[:, :nocc], ao_ovlp, verbose=verbose)

SCF not converged.
SCF energy = -18.7895418519718 after 50 cycles  <S^2> = 6.0095831  2S+1 = 5.0038318
<class 'pyscf.scf.ghf.GHF'> wavefunction has an internal instability
converged SCF energy = -18.7899070209922  <S^2> = 6.0102938  2S+1 = 5.0041158
<class 'pyscf.soscf.newton_ah.SecondOrderGHF'> wavefunction is stable in the internal stability analysis
converged SCF energy = -18.7899070210115  <S^2> = 6.0102937  2S+1 = 5.0041158
<class 'pyscf.soscf.newton_ah.SecondOrderGHF'> wavefunction is stable in the internal stability analysis
converged SCF energy = -18.7899070210146  <S^2> = 6.0102936  2S+1 = 5.0041157
<class 'pyscf.soscf.newton_ah.SecondOrderGHF'> wavefunction is stable in the internal stability analysis

# ----------------------
# Spin collinearity test
# ----------------------
# epsilon0 = 1.9999999999996207
# non integer value indicates non-collinearity

# minimum mu = 3.816669202905132e-13
# Value is 0 iff wavefunction is collinear

# If mu = 0, the collinear spin axis is: 


In [93]:
psi = jnp.array(ghf_coeff[:, :nocc])
h1 = jnp.array(h1)
chol = jnp.array(chol)
optimize(psi, n_elec, [h1, h1], chol, 0.)


# Initial projected energy = -18.79490228194406


/burg/ccce/users/su2254/envs/ad_afqmc/lib/python3.9/site-packages/jax/_src/lax/lax.py:2637: ComplexWarning: Casting complex values to real discards the imaginary part
  x_bar = _convert_element_type(x_bar, x.aval.dtype, x.aval.weak_type)


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =          192     M =         1000

At X0         0 variables are exactly at the bounds

At iterate    0    f= -1.87949D+01    |proj g|=  8.25832D-02


 This problem is unconstrained.



At iterate    1    f= -1.87970D+01    |proj g|=  3.82999D-02

At iterate    2    f= -1.87976D+01    |proj g|=  1.46080D-02

At iterate    3    f= -1.87978D+01    |proj g|=  5.05029D-03

At iterate    4    f= -1.87978D+01    |proj g|=  4.33730D-03

At iterate    5    f= -1.87978D+01    |proj g|=  4.09332D-03

At iterate    6    f= -1.87979D+01    |proj g|=  6.49331D-03

At iterate    7    f= -1.87980D+01    |proj g|=  1.07497D-02

At iterate    8    f= -1.87981D+01    |proj g|=  1.27608D-02

At iterate    9    f= -1.87982D+01    |proj g|=  7.20917D-03

At iterate   10    f= -1.87982D+01    |proj g|=  2.99731D-03

At iterate   11    f= -1.87982D+01    |proj g|=  1.20996D-03

At iterate   12    f= -1.87982D+01    |proj g|=  1.13413D-03

At iterate   13    f= -1.87982D+01    |proj g|=  1.62230D-03

At iterate   14    f= -1.87982D+01    |proj g|=  9.99091D-04

At iterate   15    f= -1.87982D+01    |proj g|=  7.03677D-04

At iterate   16    f= -1.87982D+01    |proj g|=  6.74850D-04

At iter

(-18.798225884638814,
 array([[ 0.16663,  0.18376,  0.21762,  0.1988 , -0.14757,  0.00356],
        [ 0.16282,  0.19137, -0.09728,  0.05325, -0.22921,  0.20447],
        [ 0.16599,  0.17046, -0.00006, -0.25497, -0.00001,  0.31102],
        [ 0.16822,  0.16946,  0.31821, -0.10911,  0.08396,  0.10751],
        [ 0.16801,  0.17804, -0.0986 ,  0.19978, -0.29368,  0.00359],
        [ 0.16822,  0.16946, -0.31826, -0.10893, -0.08397,  0.10752],
        [ 0.16773,  0.16481, -0.00011, -0.36233,  0.00003, -0.10515],
        [ 0.16719,  0.17486,  0.21841, -0.0519 , -0.21025, -0.20817],
        [ 0.16663,  0.18376, -0.2175 ,  0.19891,  0.14754,  0.00365],
        [ 0.16719,  0.17486, -0.21846, -0.05172,  0.21029, -0.20813],
        [ 0.1612 ,  0.19264, -0.00006, -0.13909,  0.00004, -0.31556],
        [ 0.15592,  0.22038, -0.00022,  0.10641, -0.05974, -0.10431],
        [ 0.16801,  0.17804,  0.09871,  0.19978,  0.29365,  0.00361],
        [ 0.15592,  0.22037,  0.00027,  0.10647,  0.05975, -0.10437]


           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
  192     51     58      1     0     0   1.161D-05  -1.880D+01
  F =  -18.798225884638814     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             


In [94]:
psi

Array([[ 0.16427,  0.18645,  0.20911,  0.18858, -0.1469 ,  0.00096],
       [ 0.16328,  0.19093, -0.10569,  0.06057, -0.22925,  0.20674],
       [ 0.16541,  0.17099, -0.00006, -0.25015, -0.00001,  0.31171],
       [ 0.16623,  0.1697 ,  0.31836, -0.12116,  0.08389,  0.10338],
       [ 0.1646 ,  0.18019, -0.09372,  0.1894 , -0.29313,  0.00098],
       [ 0.16623,  0.1697 , -0.31841, -0.12099, -0.08389,  0.1034 ],
       [ 0.16681,  0.16403, -0.00011, -0.37065,  0.00003, -0.10821],
       [ 0.16516,  0.17521,  0.22322, -0.05761, -0.2104 , -0.20954],
       [ 0.16427,  0.18646, -0.20899,  0.1887 ,  0.14687,  0.00103],
       [ 0.16516,  0.17521, -0.22328, -0.05743,  0.21045, -0.2095 ],
       [ 0.16256,  0.19502, -0.00006, -0.12163,  0.00003, -0.3102 ],
       [ 0.16088,  0.21886,  0.01236,  0.12063, -0.06064, -0.10062],
       [ 0.1646 ,  0.18019,  0.09384,  0.1894 ,  0.29311,  0.00101],
       [ 0.16088,  0.21885, -0.01231,  0.12069,  0.06065, -0.10065],
       [ 0.15935,  0.23152, -0.   

In [95]:
ci = fci.FCI(mol)
ci = fci.addons.fix_spin_(ci, ss=0)
e, ci_coeffs = ci.kernel(
    h1e=integrals["h1"], eri=integrals["h2"], norb=n_sites, nelec=n_elec, max_cycle=500
)

print(ci.spin_square(ci_coeffs, n_sites, n_elec))
if verbose: print(f"\n# fci energy: {e}")

(9.082898716169007e-10, 1.0000000018165798)

# fci energy: -18.83241352594775
